In [1]:
print('hello, world')

hello, world


In [1]:
#This is for figuring out the angle and sending it to unity
import cv2
import mediapipe as mp
import numpy as np
import socket

port = 690
sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM) 
serverAddressPort = ("127.0.0.1",port) 

def calculate_angle(A, B, C):
    # Form vectors AB and BC
    AB = np.array(B) - np.array(A)
    BC = np.array(C) - np.array(B)
    
    # Normalize vectors
    AB_norm = AB / np.linalg.norm(AB)
    BC_norm = BC / np.linalg.norm(BC)
    
    # Calculate cosine of the angle
    cos_angle = np.dot(AB_norm, BC_norm)
    
    # Convert cosine to angle in radians
    angle_rad = np.arccos(cos_angle)
    
    # Convert angle to degrees
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

mp_hands = mp.solutions.hands
# hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

drawingModule = mp.solutions.drawing_utils

cameraCapture = cv2.VideoCapture(0)
cv2.namedWindow('Camera Feed')

try:
    while True:
        success, frame = cameraCapture.read()
        if not success:
            break
        
        rgbImage = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgbImage)
        # print(results.multi_hand_landmarks)
        
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:       
                mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
                
                startpoint = (handLms.landmark[0].x * frame.shape[1], handLms.landmark[0].y * frame.shape[0])
                endpoint = (handLms.landmark[9].x * frame.shape[1], handLms.landmark[9].y * frame.shape[0])
                new_point = (int(startpoint[0]), int((startpoint[1]) - 50))
                cv2.circle(frame, new_point, radius=5, color=(255, 0, 0), thickness=-1)

                angle = calculate_angle(startpoint, new_point, endpoint)
                
                if startpoint[0] < endpoint[0]:
                    angle = angle * (-1)
                sock.sendto(str.encode(str(angle)),serverAddressPort)
                # print(f"The angle between AB and BC is {angle} degrees.")
                
        
        cv2.imshow('Camera Feed', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cameraCapture.release()
    cv2.destroyAllWindows()

Using left hand (but can use any hand)